# (4~10)년도 예측

In [ ]:
import os
os.chdir('./../../')
from src.basic_utils import *
from src.modeling_utils import *
from src.modeling_autoencoder import *

from sklearn.metrics.pairwise import cosine_similarity
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.initializers import glorot_normal, Zeros, Ones
import keras.backend as K
from keras.optimizers import RMSprop
import tensorflow as tf

In [ ]:
data_path = 'data/'

train = pd.read_parquet(data_path+'train(song_trim).parquet')
val = pd.read_json(data_path+'val(lower).json')
# test = pd.read_json(data_path+'test(lower).json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
# tag_song_dict = load_json(data_path+'tag_song_dict.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
# file_path = '/kaggle/input/yearly-pv-file-0-10-11-13-14-15/train_DF_%s.parquet' %small_years

small_years = [i for i in range(4,11)]
train_pv = pd.read_parquet(data_path+'train_DF_[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10].parquet')
train_pv = pd.read_parquet(data_path+'train_DF_%s.parquet' %small_years)

In [ ]:
print(train_pv.shape)
display(train_pv.head(2))
display(train_pv.tail(2))

(21630, 5719)


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


id,49,59,92,104,121,129,204,208,252,290,...,153190,153218,153221,153229,153230,153258,153313,153333,153392,153426
song,,,,,,,,,,,,,,,,,,,,,
707896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## autoencoder

In [ ]:
cf_dic = make_autoencoder(train_pv)

5719


In [ ]:
no_tag, no_song, yes_index, no_both = check_target_type(val)
v1v3_index= no_tag+yes_index

노래가 하나도 없음 :  1232
노래, 태그 둘 다 존재함 :  4190
태그가 없음 :  4507
둘다 없음 :  811


In [ ]:
train['year'] = train['updt_date'].apply( lambda x: int(x[2:4]) )
train['date'] = train['updt_date'].apply( lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply( lambda x: int(x[2:4]) )
val['date'] = val['updt_date'].apply( lambda x: int(str(x[2:4]) + str(x[5:7])) )

year_list = sorted(list(train['year'].unique()))

v1v3 = val[val.index.isin(v1v3_index)]

val_tmp = v1v3[v1v3['year'].isin(small_years)]
print("val year shape", val_tmp.shape)

val year shape (484, 8)


In [ ]:
date_list = sorted(list(train['date'].unique()))

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = make_popular_date_dict('tags', train, date_list, 13)
popular_date_song = make_popular_date_dict('songs', train, date_list, 9)

In [ ]:
v1v3_predict = pred_v1v3_auto(val_tmp, song_tag_dict, popular_date_tag)
write_json(v1v3_predict, 'v1v3_%s_predict.json' %small_years)